In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer,  
    BitsAndBytesConfig
)
from peft import  PeftModel  

/home/crysis/anaconda3/envs/gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_model = "meta-llama/Meta-Llama-3-8B"
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model, device_map="auto",  quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.22it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [4]:
eval_data_path = '../unsloth/data/val_health.jsonl'
evalset = load_dataset("json", data_files = eval_data_path, split='train')

In [5]:
path_to_adapter1 = "Crysiss/llama-3-8B-healthcare-15000-unsloth-epoch-4"
path_to_adapter2 = "Crysiss/llama-3-8B-korean-lora"
# model = PeftModel.from_pretrained(model, path_to_adapter1, adapter_name="sql")

In [6]:
model = PeftModel.from_pretrained(model, path_to_adapter1, adapter_name="sql").to('cpu')
model.load_adapter(path_to_adapter2, adapter_name="korean")

_IncompatibleKeys(missing_keys=['base_model.model.model.embed_tokens.weight', 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.sql.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.sql.weight', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.sql.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.sql.weight', 'base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.sql.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.sql.weight', 'base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.sql.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.sql.weight', 'base_model.model.model.layers.0.mlp.gate_proj.base_layer.weight', 'base_model.mod

In [7]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (sql): Identity()
                  (korean): Identity()
                )
                (lora_A): ModuleDict(
                  (sql): Linear(in_features=4096, out_features=16, bias=False)
                  (korean): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (sql): Linear(in_features=16, out_features=4096, bias=False)
                  (korean): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embe

In [8]:
model.add_weighted_adapter(["sql", "korean"], [1.0,1.0], combination_type="cat", adapter_name="sql_korean")

In [9]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (sql): Identity()
                  (korean): Identity()
                  (sql_korean): Identity()
                )
                (lora_A): ModuleDict(
                  (sql): Linear(in_features=4096, out_features=16, bias=False)
                  (korean): Linear(in_features=4096, out_features=16, bias=False)
                  (sql_korean): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (sql): Linear(in_features=16, out_features=4096, bias=Fals

In [10]:
model.delete_adapter("sql")
model.delete_adapter("korean")
# model.save_pretrained("./cat_1_1")

/home/crysis/anaconda3/envs/gpu/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:565: UserWarning: Adapter sql was active which is now deleted. Setting active adapter to korean.
  warnings.warn(
/home/crysis/anaconda3/envs/gpu/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:565: UserWarning: Adapter korean was active which is now deleted. Setting active adapter to sql_korean.
  warnings.warn(


In [11]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (sql_korean): Identity()
                )
                (lora_A): ModuleDict(
                  (sql_korean): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (sql_korean): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_fea

In [3]:
model = PeftModel.from_pretrained(model, "./cat_1_1/sql_korean")

In [12]:
korean_prompt = """당신은 AI 비서입니다. 상대방의 질문에 답변할 때 귀찮으면서 퉁명스럽게 답변해야합니다.

### 지시사항:
- 항상 답변은 한국어(korean)으로 답변합니다.

### 질문:
{}

### 답변:
{}"""

In [6]:
sql_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
- Generate a SQL query to answer

### Input:
{}

### context:
{}

### Response:
{}"""

In [18]:
i = 3

inputs = tokenizer(
[
    sql_prompt.format(
        # f"{evalset[i]['input']}", 
        # f"{evalset[i]['context']}",
        "한국어 할 수 있어?",
        "",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
result = tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [19]:
from pprint import pprint
pprint(result)

['<|begin_of_text|>Below is an instruction that describes a task, paired with '
 'an input that provides further context. Write a response that appropriately '
 'completes the request.\n'
 '\n'
 '### Instruction:\n'
 '- Generate a SQL query to answer\n'
 '\n'
 '### Input:\n'
 '한국어 할 수 있어?\n'
 '\n'
 '### context:\n'
 '\n'
 '\n'
 '### Response:\n'
 "SELECT * FROM users WHERE language = '한국어';\n"
 '\n'
 '위 쿼리를 사용하면 한국어를 할 수 있는 사용자의 정보를 가져올 수 있습니다. 이 쿼리는 users 테이블의 language 컬럼에서 '
 "'한국어'를 포함하는 레코드를 반환합니다. 결과는 사용자 ID, 이름, 이메일 주소, 전화번호 등과 같은 정보를 포함합니다. 이를 통해 "
 '한국어를 할 수 있는 사용자의 정보를 확인할 수 있습니다.<|end_of_text|>']


In [13]:
inputs = tokenizer(
[
    korean_prompt.format(
         f"{evalset[i]['input']}",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
result = tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [15]:
pprint(result)

['<|begin_of_text|>당신은 AI 비서입니다. 상대방의 질문에 답변할 때 귀찮으면서 퉁명스럽게 답변해야합니다.\n'
 '\n'
 '### 지시사항:\n'
 '- 항상 답변은 한국어(korean)으로 답변합니다.\n'
 '\n'
 '### 질문:\n'
 '한국어 할 수 있어?\n'
 '\n'
 '### 답변:\n'
 '네, 한국어를 잘 할 수 있습니다. 한국어는 우리나라의 공식 언어이며, 많은 사람들이 사용하고 있습니다. 한국어를 배우고 사용하는 것은 '
 '우리나라와 문화에 대한 이해를 높이고, 다른 사람들과 소통할 수 있는 능력을 향상시킬 수 있습니다. 한국어를 배워보시는 건 어떨까요? '
 '도움을 드리겠습니다. (한국어로 답변: 한국어를 잘 할 수 있습니다. 한국어는 우리나라의 공식 언어이며, 많은 사람들이 사용하고 '
 '있습니다. 한국어를 배우고 사용하는 것은 우리나라와 문화에 대한 이해를 높이고, 다른 사람들과 소통할 수 있는 능력을 향상시킬 수 '
 '있습니다. 한국어를 배워보시는 건 어떨까요? 도움을 드리겠습니다.) (영어로 답변: Yes, I can speak Korean. '
 'Korean is the official language of South Korea and is spoken by many people. '
 'Learning and using Korean can help you understand Korean culture and improve '
 'your communication skills. Would you like to learn Korean? I would be happy '
 'to help.) (일본어로 답변: '
 'はい、韓国語ができます。韓国語は韓国の公式言語であり、多くの人々が使用しています。韓国語を学び、使用することは韓国の文化を理解するのに役立ち、コミュニケーション能力を向上させることができます。韓国語を学びたい']
